- 해당 노트북은 "Character-Aware Neural Language Models" 논문을 기반으로 합니다.
- https://github.com/FengZiYjun/CharLM/blob/master/model.py 사이트를 참고하였습니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

from torchtext import datasets
from torchtext import data
from torchtext.data import Field, BucketIterator

import sys
sys.path.append('../ELMO/')
import string
import random
import re
import os
import time
from konlpy.tag import Okt
okt = Okt()
SEED = 1

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Declaring the Fields
- Torchtext 는 데이터를 가져오는 과정에서 선언하는 방식을 사용합니다.
    - 데이터가 어떤 형식을 지닐 것인지에 대한 것에 대해 선언을 해주고 이에 따라 torchtext 는 데이터를 로딩합니다.

## Constructing the Dataset
- fields 객체는 raw data 를 어떻게 가져올 지에 대한 선언이 담겨있습니다.
- TabularDataset 객체를 통해서, 어디서, 어떤 데이터를 가져올 지에 대해 선언을 해줍니다.
- 아래의 소스코드를 통해 형성된 객체는 generator 의 형태를 띕니다.

In [ ]:
# set up fields
TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False)

# make splits for data
train, test = datasets.IMDB.splits(TEXT, LABEL)

# build the vocabulary
TEXT.build_vocab(train)
LABEL.build_vocab(train)

# make iterator for splits
train_iter, test_iter = data.BucketIterator.splits(
    (train, test), batch_size=3, device=0)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [02:11<00:00, 640kB/s] 


- TabularDatset 을 통해서, tokenizing 까지는 되었지만, word_to_integar process는 아직 이뤄지지 않았습니다. 
- 우리의 경우, train , text 데이터 셋에 대해서 TEXT 부분에 대해서, word_to_integar converting이 필요합니다.
- `TEXT.build_vocab(trn)` 이라는 코드를 통해, converting이 가능합니다.
- 위의 연산은 모든 training set에 있는 모든 엘리먼트들을 torchtext로 만들어줍니다. Torchtext는 vocabulary를 핸들링하는 Vocab이라는 클래스를 가지고 있습니다. Vocab클래스는 word와 id를 stoi attribute에서 mapping 시켜주고, itos attribute에서는 reverse mapping시켜줍니다.
- stoi : word_to_idx default dictionary 
- itos : word list

In [9]:
batch = next(iter(train_iter))
batch.X.size()

torch.Size([15, 64])

In [11]:
word_to_idx_dict = TEXT.vocab.stoi
idx_to_word_dict = {val:idx for idx,val in word_to_idx_dict.items()}

In [33]:
word_to_idx_dict.keys()

dict_keys(['<unk>', '<pad>', '.', '이', '영화', '의', '..', '가', '에', '...', '을', '도', '들', ',', '는', '를', '은', '너무', '?', '한', '다', '정말', '만', '진짜', '적', '!', '로', '점', '으로', '에서', '평점', '연기', '것', '과', '~', '최고', '내', '그', '나', '인', '잘', '와', '안', '생각', '게', '이런', '못', '왜', '스토리', '....', '이다', '드라마', '사람', '감동', '하는', '1', '보고', '때', '더', '하고', '고', '말', '아', '감독', '배우', 'ㅋㅋ', '내용', '그냥', '거', '중', '까지', '재미', '보다', '본', '요', '!!', '없는', '좀', '뭐', '시간', '수', '지', '봤는데', '쓰레기', '사랑', '볼', '네', '작품', '다시', '하나', '없다', '10', '할', '이건', '마지막', '2', '저', '같은', '정도', '있는', 'ㅠㅠ', 'ㅋ', '좋은', '완전', '처음', '대', '장면', '주인공', '입니다', 'ㅋㅋㅋ', '이렇게', '액션', '최악', '보는', '걸', '지금', '이야기', '하', '끝', '임', '개', '3', '참', '별로', "'", '없고', '연출', '돈', '서', '느낌', '듯', '봐도', '기', '라', '재밌게', 'ㅡㅡ', '별', '역시', '인데', '명작', '난', '많이', '이해', '라고', '^^', '그리고', '면', '때문', '여자', '이영화', '보면', '해서', '전', '부터', '또', '!!!', '두', '꼭', '성', '에게', '인생', '보기', '된', '이고', '편', '아깝다', '이나', '짱', '여', '무슨', '애', '기억', '수준', '마음', '

In [17]:
','.join([idx_to_word_dict[i.item()] for i in batch.X[:,0]]).replace(",",' ')

'최고 의 드라마 ! 나가세 토모 야 연기 굿 각 키 적었지만 연기 도 굿'

In [29]:
class Highway(nn.Module):
    """Highway network"""
    def __init__(self, input_size):
        super(Highway, self).__init__()
        self.fc1 = nn.Linear(input_size, input_size, bias=True)
        self.fc2 = nn.Linear(input_size, input_size, bias=True)

    def forward(self, x):
        t = F.sigmoid(self.fc1(x))
        return torch.mul(t, F.relu(self.fc2(x))) + torch.mul(1-t, x)

class Char_CNN(nn.Module) : 
    
    def __init__(self,VOCAB_SIZE , EMBED_SIZE , HID_SIZE , DROPOUT ,KERNEL_SIZE , NUM_FILTER , N_CLASS ) : 
        super(Char_CNN, self).__init__()
        self.vocab_size = VOCAB_SIZE # character의 vocab size입니다. 논문에서는 C로 표현됩니다.
        self.embed_size = EMBED_SIZE # Embedding dimension 의 크기로 파라미터입니다.
        self.hid_size = HID_SIZE # Hidden layer의 dimension으로 이 또한 파라미터입니다.
        self.dropout = DROPOUT # dropout의 probability를 명시해줍니다.
        if type(KERNEL_SIZE) !=list :
            self.kernel_size = list(KERNEL_SIZE) # kernel의 사이즈로, 여러개의 kernel_size를 리스트 형태로 넣어줄 수 있습니다.
        else : self.kernel_size = KERNEL_SIZE # 많을 수록 complex해집니다.
        self.num_filter = NUM_FILTER # 각각의 kernel 이 몇 개씩 있는지에 대한 파라미터입니다. 많을 수록 complex해집니다.
        self.num_class = N_CLASS # output_dimension에 대한 argument로 sigmoid 값을 출력하기 때문에, 1로 합니다.
        self.device = 'cpu'
        
        self.embedding = nn.Embedding(
            num_embeddings = self.vocab_size,
            embedding_dim = self.embed_size,
            padding_idx = 1) 
        
        self.convs = nn.ModuleList([(nn.Conv2d(in_channels = 1,out_channels = self.num_filter,\
        kernel_size = (kernel,self.embed_size))) for kernel in self.kernel_size])

        
        self.fully_connect = nn.Sequential(
        nn.Linear(self.num_filter * len(self.kernel_size),self.hid_size),nn.ReLU(),
        nn.Dropout(self.dropout),nn.Linear(self.hid_size , self.num_class),
        )
    
    
    def forward(self,x) : 
        # x's dimension : [max_length, batch_size]
        if len(x.shape) == 1 :
            x.unsqueeze_(0) 
        
        embed = self.embedding(x) 
        embed = embed.unsqueeze(1) 
        embed = embed.permute(2,1,0,3)
        # [batch_size, 1 , max_length , embedding_dim]
        
        convolution = [conv(embed).squeeze(3) for conv in self.convs]
        
        
        pooled = [F.max_pool1d(conv,(conv.size(2))).squeeze(2) for conv in convolution]
        
        dropout = [F.dropout(pool,self.dropout) for pool in pooled]
        
        concatenate = torch.cat(dropout, dim = 1) 
        
        logit = self.fully_connect(concatenate)
        return logit

In [ ]:
VOCAB_SIZE = len(TEXT.vocab)
EMBED_SIZE = 256
HID_SIZE = 128
DROPOUT = 0.5
KERNEL_SIZE = [2,3,4,5]
NUM_FILTER = 4
N_CLASS = 1

model = CNN(VOCAB_SIZE, EMBED_SIZE, HID_SIZE, DROPOUT, KERNEL_SIZE, NUM_FILTER, N_CLASS)
model

In [31]:
x = batch.X[:,0]
x[0]

tensor(35)